In [1]:
#Harmonizing data
#From Aquasat GitHub
#TSS

library(readr,lib.loc = "/nas/cee-water/cjgleason/r-lib/")
library(dplyr,lib.loc = "/nas/cee-water/cjgleason/r-lib/")

ERROR: Error: package or namespace load failed for ‘readr’:
 .onLoad failed in loadNamespace() for 'readr', details:
  call: loadNamespace(name)
  error: there is no package called ‘tzdb’


In [2]:
#Define a function that renames and reorders columns from the raw files
wqp.renamer <- function(df){
  simple.names <- df %>%
                  dplyr::select(date=ActivityStartDate,
                         parameter=CharacteristicName,
                         units=ResultMeasure.MeasureUnitCode,
                         SiteID=MonitoringLocationIdentifier,
                         org=OrganizationFormalName,
                         org_id=OrganizationIdentifier,
                         time=ActivityStartTime.Time,
                         value=ResultMeasureValue,
                         sample_method=SampleCollectionMethod.MethodName,
                         analytical_method=ResultAnalyticalMethod.MethodName,
                         particle_size=ResultParticleSizeBasisText,
                         date_time=ActivityStartDateTime,
                         media=ActivityMediaName,
                         sample_depth=ActivityDepthHeightMeasure.MeasureValue,
                         sample_depth_unit=ActivityDepthHeightMeasure.MeasureUnitCode,
                         fraction=ResultSampleFractionText,
                         status=ResultStatusIdentifier) %>%
  #Remove trailing white space in labels
  mutate(units = trimws(units)) %>%
  #Keep only samples that are water samples
  filter(media=='Water')
  dropped = nrow(df)-nrow(simple.names)
  print(paste('we dropped',dropped,'samples because the sample medium was not labeled as Water'))
  return(simple.names)
}

#Function for making a nice table that gets a summary of units and the number of observations with that unit code
unit.kable <- function(d){
  d %>%
    group_by(units) %>%
    summarize(count=n()) %>%
    arrange(desc(count)) %>%
    kable(.,'html',caption='All  parameter and unit combinations') %>%
    kable_styling() %>%
    scroll_box(width='500px',height='400px')
}

unit.disharmony <- function(d,lookup){
  d %>%
    anti_join(lookup,by='units') %>%
    group_by(units) %>%
    summarize(count=n())  %>%
    kable(.,'html',caption='The following measurements
          were dropped because the units do not make sense') %>%
    kable_styling() %>%
    scroll_box(width='500px',height='400px')
}

sample.kable <- function(d){
d %>%
  group_by(sample_method) %>%
  summarize(count=n()) %>% 
  arrange(desc(count)) %>%
  kable(.,'html',caption='All sample methods and their count') %>%
  kable_styling() %>%
  scroll_box(width='600px',height='400px')
}

analytical.kable <- function(d){
  d %>%
    group_by(analytical_method) %>%
    summarize(count=n()) %>% 
    arrange(desc(count)) %>%
    kable(.,'html',caption='All analytical methods and their count') %>%
    kable_styling() %>%
    scroll_box(width='600px',height='400px')
}

In [3]:
#Read in the raw data'
tss <- readRDS("WQPdata_SNiP_HUC06_tz_full.rds") %>%
  wqp.renamer()

ERROR: Error in readRDS("WQPdata_SNiP_HUC06_tz_full.rds") %>% wqp.renamer(): could not find function "%>%"


In [4]:
#Summarize by characteristic name and unit code
unit.kable(tss)

ERROR: Error in scroll_box(., width = "500px", height = "400px"): could not find function "scroll_box"


In [6]:
analytical.kable(tss)

ERROR: Error in scroll_box(., width = "600px", height = "400px"): could not find function "scroll_box"
